In [1]:
%cd drive/MyDrive/Thesis\ Datasets

/content/drive/MyDrive/Thesis Datasets


In [3]:
!pip install pysentiment2
import pysentiment2 as ps
import csv

     |████████████████████████████████| 1.9 MB 29.6 MB/s 


In [5]:
# read in GI as key-value pairs with keys being preprocessed (famili, confid, unnecessari, etc)
import csv
import pysentiment2 as ps
import re
import pandas as pd

def read_GI(filename):
  kvp = {}
  with open(filename ,'r', ) as file:
    csvreader = csv.reader(file)
    next(csvreader)
    next(csvreader)
    for col in csvreader:
      if col[1] != "Lvd": 
        kvp[col[0]] = [str(word) for word in col[2:-2] if word]
  
  kvp2 = {}
  hiv4 = ps.HIV4()   
  for key in kvp:
    new_key = str(hiv4.tokenize(key))
    # print(new_key)
    #print(bool(new_key))
    res = re.findall(r'\w+', new_key)
    # print(res)
    if res: 
      #print(res[0])
      kvp2[res[0]] = kvp[key].copy()
    else:
      kvp2[new_key] = kvp[key].copy()
  return kvp2

key_val = read_GI("inquirerbasic_py.csv")

In [ ]:
# returning sentences and their sentiment for classification
def get_vip_sentences(dataset, sentiment, key):
  sentiments = []
  sentences = []
  sentence_tag = []
  eth = []

  with open(sentiment) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
      sentiments.append(row[0])
  with open(dataset) as file:
    hiv4 = ps.HIV4()
    csv_reader = csv.reader(file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
      text = row[2]
      #print(text)
      tokens = hiv4.tokenize(text)
      for tok in tokens:
        try:
          gi_sent = key[tok]
        except:
          print("")
       
        for gi in gi_sent:
          if gi in sentiments:
            #print(gi)
            sentences.append(text)
            sentence_tag.append(gi)
            eth.append(row[3])
            #print(gi, text) 
  
  #sentences = list(dict.fromkeys(sentences))
  with open("sent_tagged.csv", 'w', encoding='UTF8') as outfile:
    writer = csv.writer(outfile)
    header = ["Sentence", "Tag", "Race"]
    writer.writerow(header)
    for i in range(0, len(sentences)):
      writer.writerow([sentences[i], sentence_tag[i], eth[i]])
  return sentiments

sentiments = get_vip_sentences("1_sentence_data_2021.csv", "Enquirer_Comparison_filtered_sentiment.csv", key_val)

In [ ]:
len(sentiments)

56

In [ ]:
# removing duplicate rows
def rm_duplicates(file):
  df = pd.read_csv(file)
  df.drop_duplicates(inplace=True)
  df.to_csv(file, index=False)

rm_duplicates("sent_tagged.csv")

In [ ]:
# analyse each gi sentiment for W and NW to see if overall sentiment pos/neg
def get_sentiments_gi(gi_tags, dataset):
  #print(gi_tags)
  tags = []
  w_senti = []
  nw_senti = []
  for tag in gi_tags:
    w_count = 0
    nw_count = 0
    w_pol = 0
    nw_pol = 0
    hiv4 = ps.HIV4()
    with open(dataset) as file:
      csv_reader = csv.reader(file, delimiter=',')
      next(csv_reader)      
      for row in csv_reader:
        if tag.lower() == row[1].lower():
          tokens = hiv4.tokenize(row[0])
          score = hiv4.get_score(tokens)
          if row[2] == "W":
            w_count +=1
            w_pol += score['Polarity']
          else:
            nw_count +=1
            nw_pol += score['Polarity']
      print(f"\nSentiment for GI tag -> {tag}.")
      tags.append(tag)
      #print(w_pol, w_count, nw_pol, nw_count)
      w_sent = w_pol/w_count
      print(f"W sentiment {w_sent}.")
      w_senti.append(w_sent)
      nw_sent = nw_pol/nw_count
      print(f"NW sentiment {nw_sent}.")
      nw_senti.append(nw_sent)
  with open("gi_pos_neg.csv", 'w', encoding='UTF8') as outfile:
    writer = csv.writer(outfile)
    header = ["Tag", "W Sentiment", "NW Sentiment", "Difference"]
    writer.writerow(header)
    for i in range(0, len(tags)):
      writer.writerow([tags[i], w_senti[i], nw_senti[i], abs(w_senti[i]-nw_senti[i])])


get_sentiments_gi(sentiments, "sent_tagged.csv")